In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re
import pickle
from scipy import sparse as sps
from sklearn.preprocessing import normalize

In [2]:
with open ('one_grams', 'rb') as fp:
    grams = pickle.load(fp)

In [3]:
hotelReviews = pd.read_csv('C:/Users/deniz/Documents/Hotel_Reviews.csv')
nationality = ' United Kingdom '
engReviews = hotelReviews.loc[hotelReviews['Reviewer_Nationality'] == nationality][['Positive_Review','Negative_Review']]
allReviews = list(engReviews['Positive_Review'].append(engReviews['Negative_Review']))

In [4]:
def wordSplit(rev, stop_words = stopwords.words('English')):
    s = [item for sublist in [f.lower().split() for f in re.findall('\d+|\D+',rev.lower())] for item in sublist]
    s = [x for x in s if x not in stop_words]
    
    return s

def bagOfWordsShingles(review, k=3, stop_words = stopwords.words('English')):
    s = wordSplit(review, stop_words)
    if len(s) < k:
        tokens = ['']
        for s_ in s:
            tokens[0] += s_ + ' '
        tokens[0] = tokens[0][:-1]
        return tokens
        
    tokens = []
    for i in range(len(s) - k + 1):
        bag = ''
        for j in range(i, i + k):
            bag += s[j]+' '
        tokens.append(bag[:-1])
    return tokens

def bagOfWordsShingles2(review, k=3, stop_words = stopwords.words('English')):
    s = wordSplit(review, stop_words)
    tokens = []
    for K in range(0,k):
        for i in range(len(s) - (k-K) + 1):
            bag = ''
            for j in range(i, i + k-K):
                bag += s[j]+' '
            tokens.append(bag[:-1])

    return tokens

In [18]:
grams = []
for r in allReviews:
    grams.append(wordSplit(r))
    
with open('one_grams', 'wb') as fp:
    pickle.dump(grams, fp)

In [5]:
gramSet = []
for sublist in grams:
    for s in sublist:
        gramSet.append(s)
        
gramSet = list(set(gramSet))

In [6]:
gramsToIndex = dict()
i = 0
for g in gramSet:
    gramsToIndex[g] = i
    i+=1

In [12]:
transMat = sps.lil_matrix((len(gramSet), len(gramSet)))
for gram in grams:
    for i in range(len(gram) - 1):
        transMat[gramsToIndex[gram[i]], gramsToIndex[gram[i+1]]] += 1

In [13]:
normTrMat = normalize(transMat, norm = 'l1', axis = 1)

In [96]:
cState = sps.lil_matrix((1,normTrMat.shape[0])) #np.zeros(normTrMat.shape[0])
cState[0,gramsToIndex['visit']] = 1

In [97]:
predictState = cState.dot(normTrMat)
predictState = np.array(cState.dot(normTrMat).todense())[0]
probs = np.argsort(predictState)[::-1]

In [98]:
for p in probs[:5]:
    print("%s %.3f" % (gramSet[p], predictState[p]))
    #print 

london 0.086
hotel 0.052
amsterdam 0.024
staff 0.021
would 0.018
